In [3]:
# Clone YOLOv7 only once
!git clone https://github.com/WongKinYiu/yolov7
%cd yolov7

# Install required packages
!pip install -r requirements.txt


Cloning into 'yolov7'...
remote: Enumerating objects: 1197, done.
remote: Total 1197 (delta 0), reused 0 (delta 0), pack-reused 1197 (from 1)
Receiving objects: 100% (1197/1197), 74.23 MiB | 28.83 MiB/s, done.
Resolving deltas: 100% (520/520), done.
/content/yolov7
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 63.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 34.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 32.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━

In [27]:
import torch
import cv2
import pandas as pd
import os
import pickle
from models.experimental import attempt_load

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Manually load models with torch.load to avoid UnpicklingError
def safe_load(path, device):
    with open(path, 'rb') as f:
        return torch.load(f, map_location=device, pickle_module=pickle)

# Paths you gave earlier
model_paths = [
    '/content/weights/yolov7all61.pt',
    '/content/weights/yolov7all6.pt'
    # '/content/weights/mycode.pt'
]

# Pre-check existence
for p in model_paths:
    print(p, "→", os.path.exists(p))

# ✅ Load models with manual fallback
models = []
for path in model_paths:
    print(f"Loading: {path}")
    ckpt = safe_load(path, device)

    # Extract actual model if checkpoint is a dict
    if isinstance(ckpt, dict) and 'model' in ckpt:
        model = ckpt['model'].float().to(device)  # move to correct device
        model.eval()
        models.append(model)
    else:
        print(f"❌ Model not found in checkpoint: {path}")


print("✅ All models loaded successfully.")


/content/weights/yolov7all61.pt → True
/content/weights/yolov7all6.pt → True
Loading: /content/weights/yolov7all61.pt
Loading: /content/weights/yolov7all6.pt
✅ All models loaded successfully.


In [28]:
from utils.datasets import letterbox
from utils.general import non_max_suppression, scale_coords
import numpy as np
import cv2

def run_ensemble_inference(image_path, conf_threshold=0.4, iou_threshold=0.5):
    # Preprocess image
    img0 = cv2.imread(image_path)  # BGR
    assert img0 is not None, f"Image not found: {image_path}"
    img = letterbox(img0, new_shape=640)[0]
    img = img[:, :, ::-1].transpose(2, 0, 1)  # BGR to RGB, to 3x640x640
    img = np.ascontiguousarray(img)
    img = torch.from_numpy(img).to(device).float() / 255.0
    img = img.unsqueeze(0)  # add batch dimension

    all_preds = []

    for model in models:
        with torch.no_grad():
            pred = model(img, augment=False)[0]
            pred = non_max_suppression(pred, conf_thres=conf_threshold, iou_thres=iou_threshold)[0]
            if pred is not None and len(pred):
                pred[:, :4] = scale_coords(img.shape[2:], pred[:, :4], img0.shape).round()
                all_preds.append(pred)

    # Combine predictions
    if not all_preds:
        return img0, []  # No detections

    final_pred = torch.cat(all_preds, dim=0)
    final_pred = non_max_suppression(final_pred.unsqueeze(0), conf_thres=conf_threshold, iou_thres=iou_threshold)[0]

    # # Clamp coordinates to image bounds
    # h, w = img0.shape[:2]
    # final_pred[:, 0] = final_pred[:, 0].clamp(0, w)
    # final_pred[:, 1] = final_pred[:, 1].clamp(0, h)
    # final_pred[:, 2] = final_pred[:, 2].clamp(0, w)
    # final_pred[:, 3] = final_pred[:, 3].clamp(0, h)

    # Draw boxes
    for *xyxy, conf, cls in final_pred:
        label = f"{int(cls.item())} {conf:.2f}"
        cv2.rectangle(img0, (int(xyxy[0]), int(xyxy[1])), (int(xyxy[2]), int(xyxy[3])), (0, 255, 0), 2)
        cv2.putText(img0, label, (int(xyxy[0]), int(xyxy[1]) - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 1)

    return img0, final_pred.cpu().numpy()


In [29]:
output_img, preds = run_ensemble_inference("/content/test_images/test1.jpg", conf_threshold=0.4)

# Show result in Colab
from matplotlib import pyplot as plt

if output_img is not None and len(preds) > 0:
    plt.imshow(cv2.cvtColor(output_img, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.title("Detection Output")
    plt.show()
else:
    print("⚠️ No predictions made or image is empty.")


print("Prediction count:", len(preds))
print(type(output_img), output_img.shape)
print(preds[:5])


Prediction count: 1
<class 'numpy.ndarray'> (282, 391, 3)
[[     -195.5         -40       195.5         224     0.84871           0]]


In [25]:
!pip install gradio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.9/322.9 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 84.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.1 MB/s eta 0:00:00


In [30]:
import gradio as gr
import cv2
from PIL import Image
import numpy as np

def detect_from_upload(img, conf_threshold):
    # Convert to BGR and save temporarily
    img_bgr = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)
    temp_path = "/content/temp_input.jpg"
    cv2.imwrite(temp_path, img_bgr)

    # Run ensemble inference
    result_img, preds = run_ensemble_inference(temp_path, conf_threshold=conf_threshold)

    # Convert result image to RGB for display
    result_rgb = cv2.cvtColor(result_img, cv2.COLOR_BGR2RGB)
    return Image.fromarray(result_rgb)

# Define Gradio interface
gr.Interface(
    fn=detect_from_upload,
    inputs=[
        gr.Image(type="pil", label="Upload Image"),
        gr.Slider(minimum=0.1, maximum=0.9, value=0.4, label="Confidence Threshold")
    ],
    outputs=gr.Image(label="Detection Result"),
    title="YOLOv7 Ensemble Road Damage Detection",
    description="Upload an image to detect road damage using ensemble of 2 YOLOv7 models"
).launch(debug=True)


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://958d50a15dfb7b9d46.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://958d50a15dfb7b9d46.gradio.live


In [47]:
import gradio as gr
import cv2
import numpy as np
from PIL import Image

def detect_single_image(image, conf_threshold):
    img_bgr = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)
    temp_path = "/content/temp.jpg"
    cv2.imwrite(temp_path, img_bgr)
    result_img, _ = run_ensemble_inference(temp_path, conf_threshold)
    result_rgb = cv2.cvtColor(result_img, cv2.COLOR_BGR2RGB)
    return Image.fromarray(result_rgb)

def detect_multiple_images(images, conf_threshold):
    results = []
    for i, img in enumerate(images):
        img_bgr = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)
        temp_path = f"/content/temp_{i}.jpg"
        cv2.imwrite(temp_path, img_bgr)
        result_img, _ = run_ensemble_inference(temp_path, conf_threshold)
        result_rgb = cv2.cvtColor(result_img, cv2.COLOR_BGR2RGB)
        results.append(Image.fromarray(result_rgb))
    return results


In [49]:
with gr.Blocks(title="YOLOv7 Ensemble Inference") as demo:
    gr.Markdown("# 🛣️ Road Damage Detection using YOLOv7 Ensemble")
    gr.Markdown("Upload an image (or toggle multi-upload) and adjust confidence threshold.")

    conf_slider = gr.Slider(0.1, 0.9, 0.4, label="Confidence Threshold", step=0.05)
    mode = gr.Radio(["Single Image", "Multiple Images"], value="Single Image", label="Upload Mode")

    with gr.Row():
        single_input = gr.Image(type="pil", label="Upload Image")
        multi_input = gr.File(file_types=[".png", ".jpg", ".jpeg"], file_count="multiple", label="Upload Multiple Images")

    run_button = gr.Button("Run Detection")
    output_gallery = gr.Gallery(label="Result(s)")

    def route_inference(mode, image, images, conf_threshold):
        if mode == "Single Image" and image is not None:
            return [detect_single_image(image, conf_threshold)]
        elif mode == "Multiple Images" and images is not None:
            loaded_imgs = [Image.open(p.name).convert("RGB") for p in images]
            return detect_multiple_images(loaded_imgs, conf_threshold)
        return []

    run_button.click(fn=route_inference,
                     inputs=[mode, single_input, multi_input, conf_slider],
                     outputs=[output_gallery])

demo.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://d49228c51a49448c22.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
